In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
file_path = Path("C:/Users/fizza/Documents/LoanStats_2019Q1_11.csv")
df = pd.read_csv(file_path)


df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500,0.1719,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0,0,65687,38199,2000,61987,N,N
1,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1,0,271427,60641,41200,49197,N,N
2,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0,0,60644,45684,7500,43144,N,N
3,10000,0.1640,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1,0,99506,68784,19700,76506,N,N
4,22000,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0,0,219750,25919,27600,20000,N,N


In [5]:
# Convert string values to binary
df_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type", "hardship_flag", "debt_settlement_flag"], drop_first=True)
df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-19,issue_d_Mar-19,initial_list_status_w,next_pymnt_d_May-19,application_type_Joint App
0,10500,0.1719,375.35,66000.0,low_risk,27.24,0,0,8,0,...,0,0,1,1,0,0,1,1,1,0
1,25000,0.2000,929.09,105000.0,low_risk,20.23,0,0,17,1,...,1,0,0,0,1,0,1,1,1,0
2,20000,0.2000,529.88,56000.0,low_risk,24.26,0,0,8,0,...,1,0,0,0,1,0,1,1,1,0
3,10000,0.1640,353.55,92000.0,low_risk,31.44,0,1,10,1,...,0,0,1,0,1,0,1,1,1,0
4,22000,0.1474,520.39,52000.0,low_risk,18.76,0,1,14,0,...,1,0,0,0,0,0,1,1,1,0


In [6]:
# Saving the encoded dataset
# csv_path = Path("C:/Users/fizza/LoanStats_2019Q1_1.csv")
# df_encoded.to_csv(csv_path, index=False)

In [7]:
# Create our features
X = df_encoded.copy()
X = X.drop(columns=["loan_status"],axis=1)
columns_names = X.dtypes[X.dtypes=='object'].index
X = pd.get_dummies(X,columns=columns_names)


# Create our target
y = df_encoded["loan_status"]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-19,issue_d_Mar-19,initial_list_status_w,next_pymnt_d_May-19,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X.shape

(68817, 86)

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler =scaler.fit(X_train)
X_scaler

StandardScaler()

In [13]:
# Scale the training and testing data
# X_train = X_scaler.transform(X_train)
# X_test = X_scaler.transform(X_test)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(51612, 86)

In [15]:
 # Confirm scaled values...for fun
df_X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
round(df_X_train_scaled.mean(), 0)

loan_amnt                     0.0
int_rate                      0.0
installment                  -0.0
annual_inc                   -0.0
dti                           0.0
                             ... 
issue_d_Jan-19                0.0
issue_d_Mar-19                0.0
initial_list_status_w        -0.0
next_pymnt_d_May-19           0.0
application_type_Joint App    0.0
Length: 86, dtype: float64

In [16]:
 round(df_X_train_scaled.std(), 0)

loan_amnt                     1.0
int_rate                      1.0
installment                   1.0
annual_inc                    1.0
dti                           1.0
                             ... 
issue_d_Jan-19                1.0
issue_d_Mar-19                1.0
initial_list_status_w         1.0
next_pymnt_d_May-19           1.0
application_type_Joint App    1.0
Length: 86, dtype: float64

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [18]:
rf = RandomForestClassifier()
# train the model
rf.fit(X_train_scaled, y_train)
# get predictions
predictions = rf.predict(X_test)


In [19]:
# Resample the training data with the RandomForestClassifier
brf = RandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred_brf = brf.predict(X_test_scaled)
bas_brf=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_brf)

0.6829929192638629


In [21]:
# Display the confusion matrix
cm_brf = confusion_matrix(y_test, y_pred_brf)
cm_df_brf = pd.DataFrame(
    cm_brf, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_brf

,Predicted High Risk,Predicted Low Risk
Actual High Risk,37,64
Actual Low Risk,6,17098


In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.37      1.00      0.51      0.61      0.34       101
   low_risk       1.00      1.00      0.37      1.00      0.61      0.39     17104

avg / total       1.00      1.00      0.37      1.00      0.61      0.39     17205



In [23]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.0726204508847128, 'total_pymnt'),
 (0.07100942162119145, 'total_pymnt_inv'),
 (0.06656455451279822, 'total_rec_int'),
 (0.05862338020556176, 'total_rec_prncp'),
 (0.057118932205494924, 'last_pymnt_amnt'),
 (0.024743680449313066, 'installment'),
 (0.022176394598969973, 'out_prncp_inv'),
 (0.0184078484415952, 'out_prncp'),
 (0.018390711277070575, 'dti'),
 (0.017367806137256307, 'mo_sin_old_rev_tl_op'),
 (0.017063314419237443, 'mo_sin_old_il_acct'),
 (0.016788717717751304, 'max_bal_bc'),
 (0.01586468159746428, 'tot_hi_cred_lim'),
 (0.015407530660566108, 'revol_bal'),
 (0.015038814797004642, 'bc_util'),
 (0.014945141274294626, 'bc_open_to_buy'),
 (0.014233751281044222, 'tot_cur_bal'),
 (0.014051566172684675, 'avg_cur_bal'),
 (0.014000926681152557, 'loan_amnt'),
 (0.013901166004691235, 'total_bal_ex_mort'),
 (0.013478842943906891, 'annual_inc'),
 (0.01336673880650813, 'il_util'),
 (0.013294977490744332, 'total_bal_il'),
 (0.013143874057531492, 'total_il_high_credit_limit'),
 (0.01297820

In [25]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [ ]:
# Calculated the balanced accuracy score
y_pred_eec = eec.predict(X_test_scaled)
bas_eec=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_eec)

In [29]:
# Display the confusion matrix
cm_eec = confusion_matrix(y_test, y_pred_eec)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_eec

,Predicted High Risk,Predicted Low Risk
Actual High Risk,93,8
Actual Low Risk,921,16183
